## YOLO Parking Spot Detection 🚗🅿️


In [1]:
import pandas as pd
import sys
import os

# Add the path to the utils folder to the system path
if os.path.basename(os.getcwd()) == 'notebooks':
    sys.path.append('../')

from utils.functions import process_images, get_results_df, mean_df
from utils.functions import plot_model_size, plot_model_params, plot_precision_recall, plot_mAP, save_plots

# Get current working directory
repo_path = os.path.dirname(os.getcwd())

ModuleNotFoundError: No module named 'sklearn'